In [ ]:
# Clean and Transform Spotify data
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import json
import time


def cleanPlaylistData(sheetName):

  df = pd.read_excel('SpotifyPlaylistData_20221220.xlsx',sheet_name=sheetName)

  df = df.drop(columns=['Unnamed: 0','artist_uri', 'album_uri'])

  df = df.loc[:,['track_uri', 'track_name', 'artist_name', 'album_name', 'release_date', 'duration_ms', 'explicit', 'popularity']]
  # df['duration_ms'] = pd.to_timedelta(df['duration_ms'], unit='ms')
  
  return df
  
HHPlaylistDF = cleanPlaylistData('HHPlaylist')
CHHPlaylistDF = cleanPlaylistData('CHHPlaylist')


def cleanArtistData(sheetName):

  df = pd.read_excel('SpotifyPlaylistData_20221220.xlsx',sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  
  return df

def cleanAlbumData(sheetName):

  df = pd.read_excel('SpotifyPlaylistData_20221220.xlsx',sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df = df.loc[:,['album_uri', 'artist_uri', 'album_name', 'artist_name', 'release_date', 'label', 'total_tracks', 'popularity']]
  
  return df

HHAlbumsDF = cleanAlbumData('HHAlbums')

def cleanTrackFeaturesData(sheetName):

  df = pd.read_excel('SpotifyPlaylistData_20221220.xlsx',sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df.rename(columns={'uri': 'track_uri'}, inplace=True)
  
  return df

HHTrackFeaturesDF = cleanTrackFeaturesData('HHTrackFeatures')
# HHTrackFeaturesDF.head()
joinedPlaylistFeatures = HHPlaylistDF.set_index('track_uri').join(HHTrackFeaturesDF.set_index('track_uri'), lsuffix="_PL", rsuffix="_FT")
# joinedPlaylistFeatures.head()
HHPlaylistDF.head()

In [6]:
# Clean and Transform LastFM data
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import json
import time


def cleanLastFMTrackData(sheetName):

  df = pd.read_excel('LastFMData_20221220.xlsx',sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df['artist_mbid'] = df['artist_mbid'].replace(np.nan, None)
  # df = df.loc[:,['artist_mbid', 'track_name', 'artist_name', 'listeners', 'playcount']]
  # df = df.sort_values(by='artist_name', ascending=False)
  df = df.sort_values(by='playcount', ascending=False)
  df = df.reset_index(drop=True)
  
  return df

def cleanLastFMArtistData(sheetName):

  df = pd.read_excel('LastFMData_20221220.xlsx',sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  # df = df.loc[:,['artist_mbid', 'track_name', 'artist_name', 'listeners', 'playcount']]
  df = df.drop_duplicates(['artist_name'])
  df['artist_mbid'] = df['artist_mbid'].replace(np.nan, None)
  # df = df.sort_values(by='artist_name', ascending=False)
  df = df.sort_values(by='playcount', ascending=False)
  df = df.reset_index(drop=True)
  
  return df

def cleanLastFMAlbumData(sheetName):

  df = pd.read_excel('LastFMData_20221220.xlsx',sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  # df = df.loc[:,['artist_mbid', 'track_name', 'artist_name', 'listeners', 'playcount']]
  df = df.drop_duplicates(subset=['album_mbid', 'album_name'])
  df = df.sort_values(by='listeners', ascending=False)
  df['album_mbid'] = df['album_mbid'].replace(np.nan, None)
  df = df.sort_values(by='playcount', ascending=False)
  df = df.reset_index(drop=True)
  
  return df

HHAlbumData = cleanLastFMAlbumData('HHAlbumData')
CHHAlbumData = cleanLastFMAlbumData('CHHAlbumData')

HHTrackData = cleanLastFMTrackData('HHTrackData')
CHHTrackData = cleanLastFMTrackData('CHHTrackData')

HHArtistData = cleanLastFMArtistData('HHArtistData')
CHHArtistData = cleanLastFMArtistData('CHHArtistData')
# HHArtistData
# HHAlbumData
# HHTrackData

## Need to add album name to TrackData

,artist_mbid,artist_name,track_name,listeners,playcount
0,1036b808-f58c-4a3e-b461-a2c4492ecf1b,Nicki Minaj,sUPeR fREAky gIrl,358858,4852318
1,b49b81cc-d5b7-4bdd-aadb-385df8de69a6,Drake,Rich Flex,386494,2842665
2,None,LIL UZI VERT,Just Wanna Rock,243953,2246887
3,b49b81cc-d5b7-4bdd-aadb-385df8de69a6,Drake,Pussy & Millions (feat. Travis Scott),238952,1776073
4,b49b81cc-d5b7-4bdd-aadb-385df8de69a6,Drake,Major Distribution,264639,1581291
5,None,Metro Boomin,Superhero (Heroes & Villains) [with Future & C...,191145,1416049
6,None,Metro Boomin,Too Many Nights (feat. Don Toliver & with Future),170297,1131394
7,None,Metro Boomin,Niagara Falls (Foot or 2) [with Travis Scott &...,158383,1052485
8,None,Juice WRLD,In My Head,83154,989004
9,None,Glorilla,Tomorrow 2 (with Cardi B),70288,457764


In [ ]:
# Push data to excel

import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import datetime as datetime

def openFileToDataFrame(fileName):
    with open(fileName, 'r') as openfile:
        file = json.load(openfile)

    df = pd.DataFrame(file)
    return df

xlxWriter = pd.ExcelWriter('TransformedData.xlsx', engine='xlsxwriter')


xlxWriter.close()